In [326]:
# from glmhmm import *
import numpy as np
import sys
print(sys.path)
sys.path.append('/data/users/weixuan/work/model')
from fitting import *

['/data/users/weixuan/work/model/glmhmm', '/home/wliu25/miniconda3/envs/glmhmm/lib/python38.zip', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8/lib-dynload', '', '/home/wliu25/.local/lib/python3.8/site-packages', '/data/users/weixuan/work/model/ssm', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8/site-packages', '/home/wliu25/miniconda3/envs/glmhmm/lib/python3.8/site-packages/setuptools/_vendor', '/data/users/weixuan/work/model/ssm', '/data/users/weixuan/work/model', '/data/users/weixuan/work/model', '/data/users/weixuan/work/model']


In [327]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy import optimize

class GLMHMM:
    def __init__(self, N, n_states, n_features, n_outputs, max_iter=100, em_dist="gaussian"):
        """
        Initializes the GLMHMM class.

        Args:
            N (int): Number of data samples.
            n_states (int): Number of hidden states (K).
            n_features (int): Number of input features (D), excluding the bias term.
            n_outputs (int): Dimensionality of the output.
            max_iter (int): Maximum number of iterations for fitting.
            em_dist (str): Type of emission distribution ("gaussian" by default).

        Attributes:
            transition_matrix (ndarray): KxK matrix of transition probabilities.
            w (ndarray): KxDx n_outputs matrix of weights for the emission distributions.
            covariances (ndarray): Kx n_outputs x n_outputs covariance matrices for Gaussian emissions.
            pdf (callable): Probability density function for Gaussian emissions.
        """
        self.N = N
        self.n_states = n_states # K
        self.n_features = n_features + 1 # D
        self.n_outputs = n_outputs
        self.max_iter = max_iter

        # Initialize
        if em_dist == "gaussian":
            self.pdf = multivariate_normal.pdf
            # weights ~ uniform(-1, 1)
            # bias <- 1
            w = np.random.uniform(low = -1, high = 1, size = (self.n_states, self.n_features - 1, self.n_outputs))
            self.w = np.pad(w, ((0, 0), (0, 1), (0, 0)), mode='constant', constant_values=1)
            self.covariances = np.array([1e-3 * np.eye(n_outputs) for _ in range(n_states)])
        else:
            self.pdf = None
        
        self.transition_matrix = np.full((self.n_states, self.n_states), 1 / self.n_states) # uniformly distributed
        self.pi0 = (1/self.n_states) * np.ones(self.n_states)
    def dist_pdf(self, y, thetak, otherparamk=None):
        """
        Evaluates the probability density of observations for a given state.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            thetak (ndarray): Distribution parameters (e.g., mean), shape (N, output_dim).
            otherparamk (optional): Covariance matrix or other distribution-specific parameters.

        Returns:
            ndarray: Probability density values for each time step, shape (N,).
        """
        thetakt = []
        if self.pdf is not None:
            if y.ndim == 1:
                return self.pdf(y, mean=thetak, cov=otherparamk)
            
            for t in range(self.N):
                thetakt.append(self.pdf(y[t], mean=thetak[t], cov=otherparamk))
            return np.array(thetakt)
        else:
          raise Exception("No distribution function defined")
        return 0

    def dist_param(self, wk, x):
        """
        Computes the distribution parameters (e.g., the mean) for a given state and input.

        Args:
            wk (ndarray): Weights for a specific state, shape (D, output_dim).
            x (ndarray): Input data, shape (N, D).

        Returns:
            ndarray: Distribution parameters (e.g., mean) over time, shape (N, output_dim).
        """
        pre_act = (x @ wk + 1)/2
        return np.tanh(pre_act)
    
    #--------------------------------------------------------------------------------#
    #EM Algorithm
    def fit(self,y,x,A,w,pi0=None,fit_init_states=False,maxiter=250,tol=1e-3,sess=None,B=1):
        """
        Fits the GLMHMM to the data using the EM algorithm.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            x (ndarray): Input features, shape (N, D-1).
            A (ndarray): Initial transition matrix, shape (K, K).
            w (ndarray): Initial weights, shape (K, D, output_dim).
            pi0 (optional): Initial state probabilities, shape (K,).
            fit_init_states (bool): Whether to optimize initial state probabilities.
            maxiter (int): Maximum number of EM iterations.
            tol (float): Convergence tolerance for the log-likelihood.
            sess (optional): Session boundaries for separate EM computation.
            B (float): Temperature parameter for annealing.

        Returns:
            tuple:
                - Log-likelihood values for each iteration.
                - Fitted transition matrix, weights, and initial probabilities.
        """
        x = np.hstack([x, np.ones((x.shape[0], 1))])

        self.lls = np.empty(maxiter)
        self.lls[:] = np.nan

        # store variables
        self.pi0 = pi0
        
        phi = np.zeros((self.N, self.n_states))#(N, K)
        for k in range(self.n_states):
            thetak = self.dist_param(w[k], x) # calculate theta
            for t in range(self.N):
                phi[t,k] = self.dist_pdf(y[t], thetak[t], otherparamk=self.covariances[k]) # calculate phi
        # print("phi", phi)
        if sess is None:
            sess = np.array([0,self.N]) # equivalent to saying the entire data set has one session

        for n in range(maxiter):
            print(f"------------------Iter{n+1}-------------------")

            if n == maxiter - 1:
                print("Reach the max number of EM iterations")

            # E STEP
            alpha = np.zeros((self.N,self.n_states))
            beta = np.zeros_like(alpha)
            cs = np.zeros((self.N))
            self.pStates = np.zeros_like(alpha)
            self.states = np.zeros_like(cs)
            ll = 0

            for s in range(len(sess)-1): # compute E step separately over each session or day of data
                ll_s,alpha_s,_,cs_s = self.forwardPass(y[sess[s]:sess[s+1]],A,phi[sess[s]:sess[s+1],:],pi0=pi0)
                pBack_s,beta_s,zhatBack_s = self.backwardPass(y[sess[s]:sess[s+1]],A,phi[sess[s]:sess[s+1],:],alpha_s,cs_s)


                ll += ll_s
                alpha[sess[s]:sess[s+1]] = alpha_s
                cs[sess[s]:sess[s+1]] = cs_s
                self.pStates[sess[s]:sess[s+1]] = pBack_s ** B
                beta[sess[s]:sess[s+1]] = beta_s
                self.states[sess[s]:sess[s+1]] = zhatBack_s


            self.lls[n] = ll

            # M STEP
            # print("E step")
            # print("alpha", alpha)
            # print("beta", beta)
            A,w,phi,pi0 = self._updateParams(y,x,self.pStates,beta,alpha,cs,A,phi,w,fit_init_states = fit_init_states)
            # print("M step")
            print('A', A)
            print('w', w)

            # CHECK FOR CONVERGENCE
            self.lls[n] = ll
            if  n > 5 and self.lls[n-5] + tol >= ll: # break early if tolerance is reached
                break

        self.transition_matrix,self.w,self.phi,self.pi0 = A,w,phi,pi0

        return self.lls,self.transition_matrix,self.w,self.pi0
    
    # E Step
    def forwardPass(self,y,A,phi,pi0=None):
        """
        Computes forward probabilities and log-likelihood during the E-step.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            A (ndarray): Transition matrix, shape (K, K).
            phi (ndarray): Emission probabilities, shape (N, K).
            pi0 (optional): Initial state probabilities, shape (K,).

        Returns:
            tuple:
                - Log-likelihood of the observations.
                - Forward probabilities (alpha), shape (N, K).
                - Forward marginal likelihoods (cs), shape (N,).
        """

        alpha = np.zeros((y.shape[0],self.n_states)) # forward probabilities p(z_t | y_1:t)
        alpha_prior = np.zeros_like(alpha) # prior probabilities p(z_t | y_1:t-1)
        cs = np.zeros(y.shape[0]) # forward marginal likelihoods

        if not np.any(pi0):
            pi0 = np.ones(self.n_states)/self.n_states

        # first time bin
        pxz = np.multiply(phi[0,:],np.squeeze(pi0)) # weight t=0 observation probabilities by initial state probabilities
        cs[0] = np.sum(pxz) # normalizer
        # cs[0] += 1e-9

        alpha[0] = pxz/cs[0] # conditional p(z_1 | y_1)
        alpha_prior[0] = 1/self.n_states # conditional p(z_0 | y_0)

        # forward pass for remaining time bins 
        for i in np.arange(1,y.shape[0]):
            alpha_prior[i] = alpha[i-1]@A # propogate uncertainty forward
            pxz = np.multiply(phi[i,:],alpha_prior[i]) # joint P(y_1:t,z_t)
            # pxz += 1e-9
            # print('pxz',pxz)
            cs[i] = np.sum(pxz) # conditional p(y_t | y_1:t-1)
            # cs[i] += 1e-9
            alpha[i] = pxz/cs[i] # conditional p(z_t | y_1:t)
            # print("calc alpha", alpha[i])
        # print('cs', cs)
        ll = np.sum(np.log(cs))

        return ll,alpha,alpha_prior,cs


    def backwardPass(self,y,A,phi,alpha,cs):
        """
        Computes backward probabilities and posterior probabilities during the E-step.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            A (ndarray): Transition matrix, shape (K, K).
            phi (ndarray): Emission probabilities, shape (N, K).
            alpha (ndarray): Forward probabilities, shape (N, K).
            cs (ndarray): Scaling factors from forward pass, shape (N,).

        Returns:
            tuple:
                - Posterior probabilities, shape (N, K).
                - Backward probabilities (beta), shape (N, K).
                - Most probable states, shape (N,).
        """

        beta = np.zeros((y.shape[0],self.n_states))

        # last time bin
        beta[-1] = 1 # take beta(z_N) = 1

        # backward pass for remaining time bins
        '''
        In Backward Pass
        Get beta_prior beta [5.92178088e-32 6.83283147e-12]
        cs 2.0034164157341426e-09
        Calculating beta[i] [0.00170529 0.00170529]
        Get beta_prior beta [1.65544759e-23 7.40437240e-22]
        cs 2.0000000000003786e-09
        Calculating beta[i] [3.22723522e-16 3.22723522e-16]
        Get beta_prior beta [3.35131482e-22 2.12055467e-15]
        cs 2.0000010602775016e-09
        Calculating beta[i] [1.71088154e-22 1.71088154e-22]
        Get beta_prior beta [4.60877010e-26 3.03042773e-13]
        cs 2.0001515213863337e-09
        Calculating beta[i] [1.29607752e-26 1.29607752e-26]
        Get beta_prior beta [8.73985216e-139 3.58418510e-154]
        cs 2e-09
        Calculating beta[i] [2.83188149e-156 2.83188149e-156]
        Get beta_prior beta [3.37261996e-158 7.36774937e-171]
        cs 2e-09
        Calculating beta[i] [2.387715e-305 2.387715e-305]
        Get beta_prior beta [1.67971121e-140 1.41413787e-179]
        cs 2e-09
        Calculating beta[i] [0. 0.]
        '''
        # print("In Backward Pass")
        for i in np.arange(self.N-2,-1,-1):
            # print("Get beta_prior beta", phi[i+1])
            beta_prior = np.multiply(beta[i+1],phi[i+1,:]) # propogate uncertainty backward
            # beta[i] = ((A@beta_prior)+1e-9)/cs[i+1]
            beta[i] = ((A@beta_prior))/cs[i+1]
            # print('cs', cs[i+1])
            # print("Calculating beta[i]", beta[i])
            # print('cs', cs[i+1])
        # print("after backward pass")
        pBack = np.multiply(alpha,beta) # posterior after backward pass -> alpha_hat(z_n)*beta_hat(z_n)
        zhatBack = np.argmax(pBack,axis=1) # decode from likelihoods only

        # if np.round(sum(pBack[0]),5) == 1:
        #     print("Sum of posterior state probabilities does not equal 1")
        # else:
        #     print("equals 1") 

        return pBack,beta,zhatBack
    
    # M Step
    def _updateTransitions(self,y,alpha,beta,cs,A,phi):
        """
        Updates the transition probabilities during the M-step of the EM algorithm.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            alpha (ndarray): Forward probabilities, shape (N, K).
            beta (ndarray): Backward probabilities, shape (N, K).
            cs (ndarray): Scaling factors for forward probabilities, shape (N,).
            A (ndarray): Current transition matrix, shape (K, K).
            phi (ndarray): Emission probabilities, shape (N, K).

        Returns:
            ndarray: Updated transition matrix, shape (K, K).
        """

        # compute xis, the joint posterior distribution of two successive latent variables p(z_{t-1},z_t |Y,theta_old)
        xis = np.zeros((self.N-1,self.n_states,self.n_states))
        for i in np.arange(0,self.N-1):
            beta_phi = beta[i+1,:] * phi[i,:]
            alpha_reshaped = np.reshape(alpha[i,:],(self.n_states,1))
            xis[i,:,:] = ((beta_phi * alpha_reshaped) * A)/cs[i+1]

        # reshape and sum xis to obtain new transition matrix
        xis_n = np.reshape(np.sum(xis,axis=0),(self.n_states,self.n_states)) # sum_N xis
        xis_kn = np.reshape(np.sum(np.sum(xis,axis=0),axis=1),(self.n_states,1)) # sum_k sum_N xis
        A_new = xis_n/xis_kn

        return A_new

    def neglogli(self, wk, x, y, gammak, otherparamk=None, reshape_weights=False):
        """
        Computes the negative log-likelihood of observations for a specific state.

        Args:
            wk (ndarray): Weights for the current state, shape (D, output_dim).
            x (ndarray): Input data, shape (N, D).
            y (ndarray): Observations, shape (N, output_dim).
            gammak (ndarray): Posterior probabilities for the state, shape (N,).
            otherparamk (optional): Additional parameters for the distribution.
            reshape_weights (bool): If True, reshape weights to match dimensions.

        Returns:
            float: Negative log-likelihood for the state -- -sum_t (gamma_t * log(p(y_t | theta_t))). 
        """
        if reshape_weights:
            wk = wk.reshape((self.n_features, self.n_outputs))

        thetak = self.dist_param(wk, x)
        ll_list = [gammak[i] * np.log(self.dist_pdf(y[i], thetak[i], otherparamk=otherparamk)) for i in range(self.N)]
        ll = np.sum(ll_list)
        return -ll
    
    def _glmfit(self,x,wk,y,otherparamk=None, compHess=False,gammak=None,gaussianPrior=0):
        """
        Fits the GLM weights using gradient descent (L-BFGS-B).

        Args:
            x (ndarray): Input features, shape (N, D).
            wk (ndarray): Initial weights, shape (D, output_dim).
            y (ndarray): Observations, shape (N, output_dim).
            otherparamk (optional): Additional parameters for the distribution.
            compHess (bool): Compute the Hessian matrix (default=False).
            gammak (ndarray): Posterior probabilities for the state, shape (N,).
            gaussianPrior (float): Gaussian prior regularization parameter.

        Returns:
            tuple:
                - Optimized weights, shape (D, output_dim).
                - Emission probabilities, shape (N,).
        """

        w_flat = np.ndarray.flatten(wk) # flatten weights for optimization
        opt_log = lambda w: self.neglogli(w, x, y, gammak, otherparamk=otherparamk, reshape_weights=True) # calculate log likelihood

        # simplefilter(action='ignore', category=FutureWarning) # ignore FutureWarning generated by scipy
        OptimizeResult = optimize.minimize(opt_log, w_flat, jac = "True", method = "L-BFGS-B")

        wk = np.reshape(OptimizeResult.x,(self.n_features,self.n_outputs)) # reshape and update weights
        thetak = self.dist_param(wk, x) # calculate theta
        phi = self.dist_pdf(y, thetak, otherparamk=otherparamk) # calculate phi

        return wk, phi
    
    def _updateObservations(self,y,x,w,gammas):
        """
        Updates the emission parameters (weights and covariances) during the M-step.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            x (ndarray): Input features, shape (N, D).
            w (ndarray): Current weights, shape (K, D, output_dim).
            gammas (ndarray): Posterior probabilities for the states, shape (N, K).

        Returns:
            tuple:
                - Updated weights, shape (K, D, output_dim).
                - Updated emission probabilities, shape (N, K).
        """

        # reshape y from vector of indices to one-hot encoded array for matrix operations in glm.fit
        if y.ndim == 1:
          y = y.reshape((-1, 1))

        self.phi = np.zeros((self.N,self.n_states))

        for zk in np.arange(self.n_states):
            # print('zk', zk)
            self.w[zk], self.phi[:,zk] = self._glmfit(x,w[zk],y,self.covariances[zk], 
                                                      gammak=gammas[:,zk],
                                                      )

            thetak = self.dist_param(self.w[zk], x)
            residuals = y - thetak
            self.covariances[zk] = np.cov(residuals.T)
 
        return self.w, self.phi

    def _updateInitStates(self,gammas):
        """
        Updates the initial state probabilities during the M-step.

        Args:
            gammas (ndarray): Posterior probabilities for the states, shape (N, K).

        Returns:
            ndarray: Updated initial state probabilities, shape (K,).
        """
        return np.divide(gammas[0],sum(gammas[0])) # new initial latent state probabilities


    def _updateParams(self,y,x,gammas,beta,alpha,cs,A,phi,w,fit_init_states = False):
        '''
        Computes the updated parameters as part of the M-step of the EM algorithm.

        '''

        self.transition_matrix = self._updateTransitions(y,alpha,beta,cs,A,phi)

        self.w, self.phi = self._updateObservations(y,x,w,gammas)

        if fit_init_states:
            self.pi0 = self._updateInitStates(gammas)

        return self.transition_matrix, self.w, self.phi, self.pi0
    

    #----------------------------------------------------------------------#
    # Viterbi Decoding

    def _compute_likelihood(self, xt, yt):
        """
        Computes the likelihood of observations for all hidden states.

        Args:
            xt (ndarray): Input data for a single time step, shape (1, D).
            yt (ndarray): Observations for a single time step, shape (1, output_dim).

        Returns:
            ndarray: Likelihood values for all states, shape (K,).
        """

        ll = []
        for k in range(self.n_states):
            wk = self.w[k]
            thetakt = self.dist_param(wk, xt)
            ll.append(self.dist_pdf(yt, thetakt, otherparamk=self.covariances[k]))
        return np.array(ll)
        

    def mostprob_states(self, X, Y):
        """
        Decodes the most likely sequence of states using the Viterbi algorithm.

        Args:
            X (ndarray): Input data, shape (N, D).
            Y (ndarray): Observations, shape (N, output_dim).

        Returns:
            ndarray: Most likely sequence of states, shape (N,).
        """
        n_samples = X.shape[0]
        log_probs = np.zeros((n_samples, self.n_states))
        prev_states = np.zeros((n_samples, self.n_states), dtype=int)

        # Augment X with intercept column for prediction
        X_augmented = np.hstack([np.ones((X.shape[0], 1)), X])  # Add intercept column

        log_probs[0] = np.log(self._compute_likelihood(X_augmented[0], Y[0])) + np.log(1 / self.n_states)

        for t in range(1, n_samples):
            for j in range(self.n_states):
                log_probs[t, j] = np.max(log_probs[t - 1] + np.log(self.transition_matrix[:, j]) + \
                                  np.log(self._compute_likelihood(X_augmented[t], Y[t])[j]))
                prev_states[t, j] = np.argmax(log_probs[t - 1] + np.log(self.transition_matrix[:, j]) + \
                                  np.log(self._compute_likelihood(X_augmented[t], Y[t])[j]))        

        states = np.zeros(n_samples, dtype=int)
        states[-1] = np.argmax(log_probs[-1])
        # print('log_probs shape', log_probs.shape)
        # print('prev_states', prev_states)
        for t in range(n_samples - 2, -1, -1):
            # print('state', states[t+1])
            states[t] = prev_states[t + 1, states[t + 1]]

        return states
    

    #----------------------------------------------------------------------#
    # Date generation

    def generate_data(self, n_samples, X=None):
        """
        Generates synthetic data using the model's parameters.

        Args:
            n_samples (int): Number of data samples to generate.

        Returns:
            tuple: 
                - X (ndarray): Generated input features, shape (N, D-1).
                - Y (ndarray): Generated observations, shape (N, output_dim).
                - states (ndarray): True hidden states, shape (N,).
        """
        if X is None:
            X = np.random.randn(n_samples, self.n_features-1)
        states = np.zeros(n_samples, dtype=int)
        Y = np.zeros((n_samples, self.n_outputs))

        # Generate initial state and observation
        states[0] = np.random.choice(self.n_states, p=self.pi0)
        X_augmented = np.hstack([X, np.ones((X.shape[0], 1))])  # Add intercept column
        Y[0] = np.random.multivariate_normal(
            mean=self.dist_param(self.w[states[0]], X_augmented[0]),
            cov=self.covariances[states[0]]
        )

        # Generate subsequent states and observations
        for t in range(1, n_samples):
            states[t] = np.random.choice(self.n_states, p=self.transition_matrix[states[t - 1]])
            Y[t] = np.random.multivariate_normal(
                mean=self.dist_param(self.w[states[t]], X_augmented[t]),
                cov=self.covariances[states[t]]
            )

        return X, Y, states

In [293]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy import optimize
from scipy.special import logsumexp

class GLMHMM:
    def __init__(self, N, n_states, n_features, n_outputs, max_iter=100, em_dist="gaussian", l2_reg=1e-3):
        """
        Initializes the GLMHMM class.

        Args:
            N (int): Number of data samples.
            n_states (int): Number of hidden states (K).
            n_features (int): Number of input features (D), excluding the bias term.
            n_outputs (int): Dimensionality of the output.
            max_iter (int): Maximum number of iterations for fitting.
            em_dist (str): Type of emission distribution ("gaussian" by default).
            l2_reg (float): Regularization parameter for L2 penalty on weights.

        Attributes:
            transition_matrix (ndarray): KxK matrix of transition probabilities.
            w (ndarray): KxDx n_outputs matrix of weights for the emission distributions.
            covariances (ndarray): Kx n_outputs x n_outputs covariance matrices for Gaussian emissions.
            pdf (callable): Probability density function for Gaussian emissions.
        """
        self.N = N
        self.n_states = n_states  # K
        self.n_features = n_features + 1  # D (add bias term)
        self.n_outputs = n_outputs
        self.max_iter = max_iter
        self.l2_reg = l2_reg

        # Initialize
        if em_dist == "gaussian":
            self.pdf = multivariate_normal.pdf
            w = np.random.uniform(low=-1, high=1, size=(self.n_states, self.n_features - 1, self.n_outputs))
            self.w = np.pad(w, ((0, 0), (0, 1), (0, 0)), mode='constant', constant_values=1)  # Add bias term
            self.covariances = np.array([1e-3 * np.eye(n_outputs) for _ in range(n_states)])
        else:
            self.pdf = None

        self.transition_matrix = np.full((self.n_states, self.n_states), 1 / self.n_states)  # Uniformly distributed
        self.pi0 = (1 / self.n_states) * np.ones(self.n_states)

    def dist_pdf(self, y, thetak, otherparamk=None):
        """
        Evaluates the probability density of observations for a given state.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            thetak (ndarray): Distribution parameters (e.g., mean), shape (N, output_dim).
            otherparamk (optional): Covariance matrix or other distribution-specific parameters.

        Returns:
            ndarray: Probability density values for each time step, shape (N,).
        """
        if self.pdf is not None:
            if y.ndim == 1:
                return self.pdf(y, mean=thetak, cov=otherparamk)
            return np.array([self.pdf(y[t], mean=thetak[t], cov=otherparamk) for t in range(self.N)])
        raise Exception("No distribution function defined")

    def dist_param(self, wk, x):
        """
        Computes the distribution parameters (e.g., the mean) for a given state and input.

        Args:
            wk (ndarray): Weights for a specific state, shape (D, output_dim).
            x (ndarray): Input data, shape (N, D).

        Returns:
            ndarray: Distribution parameters (e.g., mean) over time, shape (N, output_dim).
        """
        pre_act = x @ wk
        return (np.tanh(pre_act) + 1) / 2

    def fit(self, y, x, A, w, pi0=None, fit_init_states=False, maxiter=250, tol=1e-3, sess=None, B=1):
        """
        Fits the GLMHMM to the data using the EM algorithm.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            x (ndarray): Input features, shape (N, D-1).
            A (ndarray): Initial transition matrix, shape (K, K).
            w (ndarray): Initial weights, shape (K, D, output_dim).
            pi0 (optional): Initial state probabilities, shape (K,).
            fit_init_states (bool): Whether to optimize initial state probabilities.
            maxiter (int): Maximum number of EM iterations.
            tol (float): Convergence tolerance for the log-likelihood.
            sess (optional): Session boundaries for separate EM computation.
            B (float): Temperature parameter for annealing.

        Returns:
            tuple:
                - Log-likelihood values for each iteration.
                - Fitted transition matrix, weights, and initial probabilities.
        """
        x = np.hstack([x, np.ones((x.shape[0], 1))])  # Add bias column

        self.lls = np.full(maxiter, np.nan)
        if pi0 is not None:
            self.pi0 = pi0

        phi = np.zeros((self.N, self.n_states))  # Emission probabilities
        for k in range(self.n_states):
            thetak = self.dist_param(w[k], x)
            phi[:, k] = self.dist_pdf(y, thetak, otherparamk=self.covariances[k])

        if sess is None:
            sess = np.array([0, self.N])  # One session spanning the whole dataset

        for n in range(maxiter):
            print(f"------------------ Iter {n + 1} -------------------")

            if n == maxiter - 1:
                print("Reached the maximum number of EM iterations.")

            # E Step
            alpha = np.zeros((self.N, self.n_states))
            beta = np.zeros_like(alpha)
            cs = np.zeros(self.N)
            self.pStates = np.zeros_like(alpha)
            self.states = np.zeros_like(cs)
            ll = 0

            for s in range(len(sess) - 1):
                ll_s, alpha_s, _, cs_s = self.forwardPass(y[sess[s]:sess[s + 1]], A, phi[sess[s]:sess[s + 1], :], pi0=self.pi0)
                pBack_s, beta_s, zhatBack_s = self.backwardPass(y[sess[s]:sess[s + 1]], A, phi[sess[s]:sess[s + 1], :], alpha_s, cs_s)

                ll += ll_s
                alpha[sess[s]:sess[s + 1]] = alpha_s
                cs[sess[s]:sess[s + 1]] = cs_s
                self.pStates[sess[s]:sess[s + 1]] = pBack_s ** B
                beta[sess[s]:sess[s + 1]] = beta_s
                self.states[sess[s]:sess[s + 1]] = zhatBack_s

            self.lls[n] = ll

            # M Step
            w_old = self.w.copy()
            A, w, phi, self.pi0 = self._updateParams(y, x, self.pStates, beta, alpha, cs, A, phi, w, fit_init_states=fit_init_states)
            print('Updated Transition Matrix:', A)

            # Convergence Check
            param_change = np.linalg.norm(self.w - w_old)
            print(f'Parameter Change: {param_change}')
            if param_change < tol:
                break

        self.transition_matrix, self.w, self.phi, self.pi0 = A, w, phi, self.pi0

        return self.lls, self.transition_matrix, self.w, self.pi0

    def forwardPass(self, y, A, phi, pi0=None):
        """
        Computes forward probabilities and log-likelihood during the E-step.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            A (ndarray): Transition matrix, shape (K, K).
            phi (ndarray): Emission probabilities, shape (N, K).
            pi0 (optional): Initial state probabilities, shape (K,).

        Returns:
            tuple:
                - Log-likelihood of the observations.
                - Forward probabilities (alpha), shape (N, K).
                - Forward marginal likelihoods (cs), shape (N,).
        """
        alpha = np.zeros((y.shape[0], self.n_states))
        cs = np.zeros(y.shape[0])

        if pi0 is None:
            pi0 = np.ones(self.n_states) / self.n_states

        # First time step
        log_pxz = np.log(phi[0, :]) + np.log(pi0)
        cs[0] = logsumexp(log_pxz)
        alpha[0] = np.exp(log_pxz - cs[0])

        # Remaining time steps
        for i in range(1, y.shape[0]):
            log_pxz = np.log(phi[i, :]) + logsumexp(np.log(A.T) + np.log(alpha[i - 1]))
            cs[i] = logsumexp(log_pxz)
            alpha[i] = np.exp(log_pxz - cs[i])

        ll = np.sum(cs)
        return ll, alpha, None, cs

    def backwardPass(self, y, A, phi, alpha, cs):
        """
        Computes backward probabilities and posterior probabilities during the E-step.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            A (ndarray): Transition matrix, shape (K, K).
            phi (ndarray): Emission probabilities, shape (N, K).
            alpha (ndarray): Forward probabilities, shape (N, K).
            cs (ndarray): Scaling factors from forward pass, shape (N,).

        Returns:
            tuple:
                - Posterior probabilities, shape (N, K).
                - Backward probabilities (beta), shape (N, K).
                - Most probable states, shape (N,).
        """
        beta = np.zeros((y.shape[0], self.n_states))
        beta[-1] = 1

        # Backward pass
        for i in range(y.shape[0] - 2, -1, -1):
            beta_prior = np.multiply(beta[i + 1], phi[i + 1, :])
            beta[i] = (A @ beta_prior) / cs[i + 1]

        pBack = np.multiply(alpha, beta)
        zhatBack = np.argmax(pBack, axis=1)
        return pBack, beta, zhatBack

    def _updateTransitions(self, y, alpha, beta, cs, A, phi):
        """
        Updates the transition probabilities during the M-step of the EM algorithm.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            alpha (ndarray): Forward probabilities, shape (N, K).
            beta (ndarray): Backward probabilities, shape (N, K).
            cs (ndarray): Scaling factors for forward probabilities, shape (N,).
            A (ndarray): Current transition matrix, shape (K, K).
            phi (ndarray): Emission probabilities, shape (N, K).

        Returns:
            ndarray: Updated transition matrix, shape (K, K).
        """
        xis = np.zeros((self.N - 1, self.n_states, self.n_states))
        for i in range(self.N - 1):
            beta_phi = beta[i + 1, :] * phi[i, :]
            alpha_reshaped = np.reshape(alpha[i, :], (self.n_states, 1))
            xis[i, :, :] = ((beta_phi * alpha_reshaped) * A) / cs[i + 1]

        xis_n = np.sum(xis, axis=0)
        xis_kn = np.sum(xis_n, axis=1, keepdims=True)
        A_new = xis_n / xis_kn
        return A_new

    def neglogli(self, wk, x, y, gammak, otherparamk=None, reshape_weights=False):
        """
        Computes the negative log-likelihood of observations for a specific state.

        Args:
            wk (ndarray): Weights for the current state, shape (D, output_dim).
            x (ndarray): Input data, shape (N, D).
            y (ndarray): Observations, shape (N, output_dim).
            gammak (ndarray): Posterior probabilities for the state, shape (N,).
            otherparamk (optional): Additional parameters for the distribution.
            reshape_weights (bool): If True, reshape weights to match dimensions.

        Returns:
            float: Negative log-likelihood for the state -- -sum_t (gamma_t * log(p(y_t | theta_t))). 
        """
        if reshape_weights:
            wk = wk.reshape((self.n_features, self.n_outputs))

        thetak = self.dist_param(wk, x)
        ll_list = [gammak[i] * np.log(self.dist_pdf(y[i], thetak[i], otherparamk=otherparamk)) for i in range(self.N)]
        reg_penalty = self.l2_reg * np.sum(wk**2)
        ll = np.sum(ll_list)
        return -(ll - reg_penalty)

    def _glmfit(self, x, wk, y, otherparamk=None, gammak=None):
        """
        Fits the GLM weights using gradient descent (L-BFGS-B).

        Args:
            x (ndarray): Input features, shape (N, D).
            wk (ndarray): Initial weights, shape (D, output_dim).
            y (ndarray): Observations, shape (N, output_dim).
            otherparamk (optional): Additional parameters for the distribution.
            gammak (ndarray): Posterior probabilities for the state, shape (N,).

        Returns:
            tuple:
                - Optimized weights, shape (D, output_dim).
                - Emission probabilities, shape (N,).
        """
        w_flat = np.ndarray.flatten(wk)
        opt_log = lambda w: self.neglogli(w, x, y, gammak, otherparamk=otherparamk, reshape_weights=True)
        OptimizeResult = optimize.minimize(opt_log, w_flat, method="CG") #L-BFGS-B

        if not OptimizeResult.success:
            print("Optimization failed:", OptimizeResult.message)

        wk = np.reshape(OptimizeResult.x, (self.n_features, self.n_outputs))
        thetak = self.dist_param(wk, x)
        phi = self.dist_pdf(y, thetak, otherparamk=otherparamk)
        return wk, phi

    def _updateObservations(self, y, x, w, gammas):
        """
        Updates the emission parameters (weights and covariances) during the M-step.

        Args:
            y (ndarray): Observations, shape (N, output_dim).
            x (ndarray): Input features, shape (N, D).
            w (ndarray): Current weights, shape (K, D, output_dim).
            gammas (ndarray): Posterior probabilities for the states, shape (N, K).

        Returns:
            tuple:
                - Updated weights, shape (K, D, output_dim).
                - Updated emission probabilities, shape (N, K).
        """
        self.phi = np.zeros((self.N, self.n_states))

        for zk in range(self.n_states):
            w[zk], self.phi[:, zk] = self._glmfit(x, w[zk], y, gammak=gammas[:, zk])
            residuals = y - self.dist_param(w[zk], x)
            self.covariances[zk] = np.cov(residuals.T) + 1e-6 * np.eye(self.n_outputs)  # Regularize covariances

        return w, self.phi

    def _updateInitStates(self, gammas):
        """
        Updates the initial state probabilities during the M-step.

        Args:
            gammas (ndarray): Posterior probabilities for the states, shape (N, K).

        Returns:
            ndarray: Updated initial state probabilities, shape (K,).
        """
        return np.divide(gammas[0], np.sum(gammas[0]))

    def _updateParams(self, y, x, gammas, beta, alpha, cs, A, phi, w, fit_init_states=False):
        """
        Computes the updated parameters as part of the M-step of the EM algorithm.
        """
        self.transition_matrix = self._updateTransitions(y, alpha, beta, cs, A, phi)
        self.w, self.phi = self._updateObservations(y, x, w, gammas)
        if fit_init_states:
            self.pi0 = self._updateInitStates(gammas)

        return self.transition_matrix, self.w, self.phi, self.pi0

    #----------------------------------------------------------------------#
    # Viterbi Decoding

    def _compute_likelihood(self, xt, yt):
        """
        Computes the likelihood of observations for all hidden states.

        Args:
            xt (ndarray): Input data for a single time step, shape (1, D).
            yt (ndarray): Observations for a single time step, shape (1, output_dim).

        Returns:
            ndarray: Likelihood values for all states, shape (K,).
        """

        ll = []
        for k in range(self.n_states):
            wk = self.w[k]
            thetakt = self.dist_param(wk, xt)
            ll.append(self.dist_pdf(yt, thetakt, otherparamk=self.covariances[k]))
        return np.array(ll)
        

    def mostprob_states(self, X, Y):
        """
        Decodes the most likely sequence of states using the Viterbi algorithm.

        Args:
            X (ndarray): Input data, shape (N, D).
            Y (ndarray): Observations, shape (N, output_dim).

        Returns:
            ndarray: Most likely sequence of states, shape (N,).
        """
        n_samples = X.shape[0]
        log_probs = np.zeros((n_samples, self.n_states))
        prev_states = np.zeros((n_samples, self.n_states), dtype=int)

        # Augment X with intercept column for prediction
        X_augmented = np.hstack([np.ones((X.shape[0], 1)), X])  # Add intercept column

        log_probs[0] = np.log(self._compute_likelihood(X_augmented[0], Y[0])) + np.log(1 / self.n_states)

        for t in range(1, n_samples):
            for j in range(self.n_states):
                log_probs[t, j] = np.max(log_probs[t - 1] + np.log(self.transition_matrix[:, j]) + \
                                  np.log(self._compute_likelihood(X_augmented[t], Y[t])[j]))
                prev_states[t, j] = np.argmax(log_probs[t - 1] + np.log(self.transition_matrix[:, j]) + \
                                  np.log(self._compute_likelihood(X_augmented[t], Y[t])[j]))        

        states = np.zeros(n_samples, dtype=int)
        states[-1] = np.argmax(log_probs[-1])
        # print('log_probs shape', log_probs.shape)
        # print('prev_states', prev_states)
        for t in range(n_samples - 2, -1, -1):
            # print('state', states[t+1])
            states[t] = prev_states[t + 1, states[t + 1]]

        return states
    

    #----------------------------------------------------------------------#
    # Date generation

    def generate_data(self, n_samples, X=None):
        """
        Generates synthetic data using the model's parameters.

        Args:
            n_samples (int): Number of data samples to generate.

        Returns:
            tuple: 
                - X (ndarray): Generated input features, shape (N, D-1).
                - Y (ndarray): Generated observations, shape (N, output_dim).
                - states (ndarray): True hidden states, shape (N,).
        """
        if X is None:
            X = np.random.randn(n_samples, self.n_features-1)
        states = np.zeros(n_samples, dtype=int)
        Y = np.zeros((n_samples, self.n_outputs))

        # Generate initial state and observation
        states[0] = np.random.choice(self.n_states, p=self.pi0)
        X_augmented = np.hstack([X, np.ones((X.shape[0], 1))])  # Add intercept column
        Y[0] = np.random.multivariate_normal(
            mean=self.dist_param(self.w[states[0]], X_augmented[0]),
            cov=self.covariances[states[0]]
        )

        # Generate subsequent states and observations
        for t in range(1, n_samples):
            states[t] = np.random.choice(self.n_states, p=self.transition_matrix[states[t - 1]])
            Y[t] = np.random.multivariate_normal(
                mean=self.dist_param(self.w[states[t]], X_augmented[t]),
                cov=self.covariances[states[t]]
            )

        return X, Y, states

In [209]:
# init = 3
# lls_all = np.zeros((init, 250))
# A_all = np.zeros((init, 2, 2))
# w_all = np.zeros((init, 2, 2, 1))
# pi0_all = np.zeros((init, 2))
# for i in range(init):
#   print(f'-----------------init {i}-----------------')
#   model_pred = GLMHMM(num_data, 2, 1, 1)
#   A=model_pred.transition_matrix
#   w=model_pred.w
#   lls,A,w,pi0 = model_pred.fit(Y,X,A,w, fit_init_states=True)
#   lls_all[i] = lls
#   A_all[i] = A
#   w_all[i] = w
#   pi0_all[i] = pi0

In [210]:
# hmm = ssm.HMM(2, 1, M=1, transitions="inputdriven")
# hmm_lls = hmm.fit(Y, inputs=X, method="em", num_iters=100, init_method="kmeans")

NameError: name 'ssm' is not defined

In [ ]:
# hmm.transitions.Ws, np.exp(hmm.transitions.log_Ps)

In [ ]:
# hmm = ssm.HMM(2, 1)
# hmm_lls = hmm.fit(Y, method="em", num_iters=100, init_method="kmeans")
# hmm.transitions.transition_matrix

In [ ]:
def find_best_fit(lls):
    cleaned_lls = lls[~np.isnan(lls).all(axis=1)]
    assert len(cleaned_lls) != 0
    return np.argmax(np.nanmax(cleaned_lls,axis=1))

In [ ]:
# bestidx = find_best_fit(lls_all) # run several times (several inti) and get the best one
# A_pred = A_all[bestidx]
# w_pred = w_all[bestidx]
# pi0_pred = pi0_all[bestidx]

AssertionError: 

In [ ]:
model_true = GLMHMM(num_data, 2, 1, 1) #N, n_states, n_features, n_outputs
model_true.transition_matrix = np.array([[0.5, 0.5], [0.5, 0.5]])

In [ ]:
#N = 500, 1000, n_state = 3, n_features(x_d)=?4?7? same as agent, n_output(dim(y))=2(same as agent)
# w is of shape self.n_states, self.n_features + 1, self.n_outputs
# w in R^{n_features + 1 x n_states}

# Easy Level (2 hidden states)

In [304]:
N=500
K=2
D=2
dim_output=2

In [305]:
div_pt = np.linspace(-1, 1, K+1)
w_true = np.zeros((K, D, dim_output))
for d in range(D):
    #k=0 -> neg
    #k=1 -> pos
    for k in range(K):
        w_true[k, d, :] = np.random.uniform(low=div_pt[k], high=div_pt[k+1], \
                size = dim_output)

# bias <- 0s
w_true = np.pad(w_true, ((0, 0), (0, 1), (0, 0)), mode='constant')
w_true

array([[[-0.37953102, -0.4780201 ],
        [-0.58792056, -0.36386419],
        [ 0.        ,  0.        ]],

       [[ 0.1581271 ,  0.39070141],
        [ 0.99147272,  0.88452461],
        [ 0.        ,  0.        ]]])

In [315]:
model_true = GLMHMM(N, K, D, dim_output) #N, n_states, n_features, n_outputs
# A_true = np.array([[0.8, 0.2], [0.2, 0.8]])
# model_true.transition_matrix = A_true
# model_true.w = w_true

A_true = model_true.transition_matrix
w_true = model_true.w

# X = np.ones((N, D))
X, Y, states = model_true.generate_data(N)

In [307]:
Y, states

(array([[ 0.83745635,  0.85951231],
        [ 0.77698047,  0.95466553],
        [ 0.60309351,  0.91695686],
        [ 0.48408428,  0.87392212],
        [ 0.76137013,  0.01780564],
        [ 0.87368226,  0.48119654],
        [ 0.39683674,  0.91229098],
        [ 0.94333423,  0.66963389],
        [ 0.9253367 ,  0.72685458],
        [ 0.57841292,  0.6743618 ],
        [ 0.79527295,  0.60821068],
        [ 0.91317646,  0.77936085],
        [-0.79309005,  1.01987919],
        [ 0.98259444,  0.88511918],
        [ 0.58944693,  0.71977146],
        [ 0.7876367 ,  0.84835988],
        [ 0.2380034 ,  0.73621021],
        [ 0.41315447,  0.66876824],
        [ 0.85203172,  0.56052872],
        [ 0.81236787,  0.81832309],
        [ 0.07377704,  0.96938157],
        [ 0.94625228,  0.75291994],
        [ 0.36456922,  0.77507859],
        [ 1.03023673,  0.80632813],
        [ 0.85905225,  0.71849905],
        [ 0.55269529,  0.86676483],
        [ 0.98571664,  0.68041547],
        [ 0.34586335,  0.799

In [308]:
phi = np.zeros((N, K))#(N, K)
X_augmented = np.hstack([X, np.ones((X.shape[0], 1))])
for k in range(K):
    thetak = model_true.dist_param(w_true[k], X_augmented) # calculate theta
    print(thetak)
    for t in range(N):
        phi[t,k] = model_true.dist_pdf(Y[t], thetak[t], otherparamk=model_true.covariances[k]) # calculate phi


[[ 8.67872140e-01  8.73743610e-01]
 [ 7.76532105e-01  9.49448897e-01]
 [ 5.94053601e-01  9.01119179e-01]
 [ 5.04099611e-01  9.33371860e-01]
 [ 8.10769101e-01 -1.71984141e-02]
 [ 8.67324642e-01  5.05105286e-01]
 [ 4.40152526e-01  8.70552209e-01]
 [ 7.22061653e-02  9.90721920e-01]
 [ 5.90935042e-01  9.59146299e-01]
 [ 5.82831880e-01  6.37505444e-01]
 [ 8.00963935e-01  5.92216207e-01]
 [ 9.27470902e-01  5.41005304e-01]
 [-7.83240462e-01  9.82588056e-01]
 [ 9.88516567e-01 -7.04786236e-02]
 [ 6.15819223e-01  8.28791472e-01]
 [ 9.91813040e-01 -8.31056046e-01]
 [ 2.83580489e-01  8.86337640e-01]
 [-3.65295457e-01  9.82562409e-01]
 [-6.02908354e-01  9.96269481e-01]
 [ 8.08733271e-01  8.10990608e-01]
 [ 6.91696025e-02  9.77836270e-01]
 [ 9.08432792e-01  6.83866041e-01]
 [ 7.56406397e-01  5.70681616e-01]
 [ 9.71089763e-01  7.06054085e-01]
 [ 5.95246197e-01  9.15696126e-01]
 [ 5.93599959e-01  8.62938411e-01]
 [ 6.36395233e-01  9.59956112e-01]
 [ 7.93785617e-01  4.41564301e-01]
 [ 8.55261942e-01  7

In [309]:
phi

array([[9.05636615e+001, 1.17344914e-005],
       [1.56988264e+002, 3.74962880e-021],
       [1.34774389e+002, 5.52228964e-016],
       [2.22520812e+001, 2.07381827e-031],
       [2.54599620e+001, 0.00000000e+000],
       [1.17196797e+002, 9.50663296e-024],
       [2.60674455e+001, 2.98044253e-006],
       [1.06961809e-185, 6.71064331e+001],
       [1.59331348e-034, 6.77198119e+001],
       [7.99114511e+001, 9.61674414e-083],
       [1.37795820e+002, 2.57199324e-013],
       [6.61582380e-011, 1.43292588e+002],
       [7.56453901e+001, 1.92288285e-031],
       [7.98863679e-197, 2.96443088e+001],
       [2.95067078e-001, 4.38801872e+001],
       [0.00000000e+000, 2.35639751e+001],
       [7.18834293e-004, 1.30630569e+002],
       [1.70691779e-151, 9.57862982e+001],
       [0.00000000e+000, 2.64286110e+001],
       [1.53913444e+002, 6.88080032e-001],
       [1.51945668e+002, 9.40177133e-140],
       [7.17426925e+000, 1.24776496e+002],
       [6.16334670e-041, 8.98611196e+001],
       [1.8

In [316]:
ll_s,alpha_s,_,cs_s = model_true.forwardPass(Y,A_true,phi,pi0=model_true.pi0)
# ll_s, alpha_s, cs_s
ll_s

1732.578558988474

In [317]:
model_pred = GLMHMM(N, K, D, dim_output)
A=model_pred.transition_matrix
w=model_pred.w
A, w

(array([[0.5, 0.5],
        [0.5, 0.5]]),
 array([[[-0.65806885, -0.27761481],
         [ 0.99373737,  0.08438766],
         [ 1.        ,  1.        ]],
 
        [[-0.10761315, -0.51203473],
         [-0.95103289,  0.68942621],
         [ 1.        ,  1.        ]]]))

if /tmp/ipykernel_545288/3460029136.py:335: RuntimeWarning: divide by zero encountered in log
  ll_list = [gammak[i] * np.log(self.dist_pdf(y[i], thetak[i], otherparamk=otherparamk)) for i in range(self.N)]
/tmp/ipykernel_545288/3460029136.py:335: RuntimeWarning: invalid value encountered in scalar multiply
  ll_list = [gammak[i] * np.log(self.dist_pdf(y[i], thetak[i], otherparamk=otherparamk)) for i in range(self.N)] is triggered, all K-1 states become trivial

if ------------------Iter1-------------------
/tmp/ipykernel_545288/637519072.py:209: RuntimeWarning: invalid value encountered in divide
  alpha[i] = pxz/cs[i] # conditional p(z_t | y_1:t)
/tmp/ipykernel_545288/637519072.py:212: RuntimeWarning: divide by zero encountered in log
  ll = np.sum(np.log(cs))
/tmp/ipykernel_545288/637519072.py:335: RuntimeWarning: divide by zero encountered in log
  ll_list = [gammak[i] * np.log(self.dist_pdf(y[i], thetak[i], otherparamk=otherparamk)) for i in range(self.N)], then transition matrix become nans

In [318]:
lls,A,w,pi0 = model_pred.fit(Y,X,A,w, fit_init_states=True) 

------------------Iter1-------------------


/tmp/ipykernel_545288/637519072.py:209: RuntimeWarning: invalid value encountered in divide
  alpha[i] = pxz/cs[i] # conditional p(z_t | y_1:t)
/tmp/ipykernel_545288/637519072.py:212: RuntimeWarning: divide by zero encountered in log
  ll = np.sum(np.log(cs))
/tmp/ipykernel_545288/637519072.py:335: RuntimeWarning: divide by zero encountered in log
  ll_list = [gammak[i] * np.log(self.dist_pdf(y[i], thetak[i], otherparamk=otherparamk)) for i in range(self.N)]


A [[nan nan]
 [nan nan]]
w [[[-0.65806885 -0.27761481]
  [ 0.99373737  0.08438766]
  [ 1.          1.        ]]

 [[-0.10761315 -0.51203473]
  [-0.95103289  0.68942621]
  [ 1.          1.        ]]]
------------------Iter2-------------------


KeyboardInterrupt: 

In [257]:
lls

array([-23235.42389814, -14202.64280142,    -49.38408035,    613.92566279,
          694.70013304,    695.05173349,    695.10746282,    695.12818468,
          695.13420327,    695.13631618,    695.13706286,    695.13732998,
          695.1374266 ,    695.13747024,    695.13751265,    695.13759558,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,  

In [247]:
A, model_true.transition_matrix

(array([[0.9983317 , 0.0016683 ],
        [0.01038264, 0.98961736]]),
 array([[0.5, 0.5],
        [0.5, 0.5]]))

In [248]:
w_true, w

(array([[[ 0.94585715, -0.66548002],
         [ 0.58126591,  0.55659694],
         [ 1.        ,  1.        ]],
 
        [[-0.98796259, -0.24798168],
         [ 0.00949847,  0.38307131],
         [ 1.        ,  1.        ]]]),
 array([[[-0.70712947, -0.27107093],
         [ 0.16068875,  0.3903825 ],
         [ 1.00525131,  0.90780638]],
 
        [[ 0.33950318, -0.61739619],
         [ 0.30888886,  0.54245452],
         [ 0.85033738,  1.02021792]]]))

In [249]:
model_pred.mostprob_states(X, Y), states

(array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 

In [ ]:
N=500
K=3
D=4
dim_output=2



In [ ]:
for k in 

# Testing pipeline: true # of states vs best # states 

In [321]:
# Given Y, X
true_states = np.arange(5)+1 #1-5
for true_state in true_states:
    val_lls, aris, res_params = OptStateCV_traj(Y, \
        model = "glmhmm", states_cands=[1,2,3,4,5], \
        inpts=X, num_init=3, obs_dist="gaussian", verbose=False)

    # val_lls: (len(states_cands), n_folds, num_init)  # array to store performance results
    # aris: (len(states_cands), n_folds)
    # res_params[num_states]['ws'], res_params[num_states]["As"], res_params[num_states]["pi0"]

    # TODO: Choose the optimal # of states

    

array([0, 1, 2, 3, 4])